<a href="https://colab.research.google.com/github/marcosvandrade/dataengineer-XPe-IGTI/blob/main/Aula2_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
import os

In [5]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
dir = '/content/drive/MyDrive/xpe/data'

list_files = os.listdir(dir)
list_files

['2022_Passagem.csv', 'compras.csv']

In [8]:
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=1c05a7006f96c023f34133cafc190d6ed6a0f89e6b909c58bf8aae39326c4345
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [9]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Aula2.5").getOrCreate()

In [12]:
from IPython.utils.sysinfo import encoding

csv_file = "2022_Passagem.csv"
df = spark.read.csv(f"{dir}/{csv_file}", header = True, encoding="iso-8859-1", sep=";")

df.show(5)

+-----------------------------------+-------------------------+------------------+-----------------+---------------+-------------------+------------------+----------------+--------------------+-------------------+-----------------+---------------------+--------------------+------------------+----------------------+-----------------+---------------+----------------------+----------------------+
|Identificador do processo de viagem|Número da Proposta (PCDP)|Meio de transporte|País - Origem ida|UF - Origem ida|Cidade - Origem ida|País - Destino ida|UF - Destino ida|Cidade - Destino ida|País - Origem volta|UF - Origem volta|Cidade - Origem volta|Pais - Destino volta|UF - Destino volta|Cidade - Destino volta|Valor da passagem|Taxa de serviço|Data da emissão/compra|Hora da emissão/compra|
+-----------------------------------+-------------------------+------------------+-----------------+---------------+-------------------+------------------+----------------+--------------------+-------------

In [13]:
df.printSchema()

root
 |-- Identificador do processo de viagem: string (nullable = true)
 |-- Número da Proposta (PCDP): string (nullable = true)
 |-- Meio de transporte: string (nullable = true)
 |-- País - Origem ida: string (nullable = true)
 |-- UF - Origem ida: string (nullable = true)
 |-- Cidade - Origem ida: string (nullable = true)
 |-- País - Destino ida: string (nullable = true)
 |-- UF - Destino ida: string (nullable = true)
 |-- Cidade - Destino ida: string (nullable = true)
 |-- País - Origem volta: string (nullable = true)
 |-- UF - Origem volta: string (nullable = true)
 |-- Cidade - Origem volta: string (nullable = true)
 |-- Pais - Destino volta: string (nullable = true)
 |-- UF - Destino volta: string (nullable = true)
 |-- Cidade - Destino volta: string (nullable = true)
 |-- Valor da passagem: string (nullable = true)
 |-- Taxa de serviço: string (nullable = true)
 |-- Data da emissão/compra: string (nullable = true)
 |-- Hora da emissão/compra: string (nullable = true)



In [14]:
import pyspark.sql.functions as F

df_ = df.select([F.col(x).alias(x.replace("-","").replace(" ", "_").lower()) for x in df.columns])

df_

DataFrame[identificador_do_processo_de_viagem: string, número_da_proposta_(pcdp): string, meio_de_transporte: string, país__origem_ida: string, uf__origem_ida: string, cidade__origem_ida: string, país__destino_ida: string, uf__destino_ida: string, cidade__destino_ida: string, país__origem_volta: string, uf__origem_volta: string, cidade__origem_volta: string, pais__destino_volta: string, uf__destino_volta: string, cidade__destino_volta: string, valor_da_passagem: string, taxa_de_serviço: string, data_da_emissão/compra: string, hora_da_emissão/compra: string]

In [15]:
df_.printSchema()

root
 |-- identificador_do_processo_de_viagem: string (nullable = true)
 |-- número_da_proposta_(pcdp): string (nullable = true)
 |-- meio_de_transporte: string (nullable = true)
 |-- país__origem_ida: string (nullable = true)
 |-- uf__origem_ida: string (nullable = true)
 |-- cidade__origem_ida: string (nullable = true)
 |-- país__destino_ida: string (nullable = true)
 |-- uf__destino_ida: string (nullable = true)
 |-- cidade__destino_ida: string (nullable = true)
 |-- país__origem_volta: string (nullable = true)
 |-- uf__origem_volta: string (nullable = true)
 |-- cidade__origem_volta: string (nullable = true)
 |-- pais__destino_volta: string (nullable = true)
 |-- uf__destino_volta: string (nullable = true)
 |-- cidade__destino_volta: string (nullable = true)
 |-- valor_da_passagem: string (nullable = true)
 |-- taxa_de_serviço: string (nullable = true)
 |-- data_da_emissão/compra: string (nullable = true)
 |-- hora_da_emissão/compra: string (nullable = true)



In [17]:
df_1 = df_.withColumn("valor_da_passagem", F.expr("cast(replace(valor_da_passagem, ',', '.') as float)"))\
          .withColumn("taxa_de_serviço", F.expr("cast(replace(`taxa_de_serviço`, ',', '.') as float)"))

df_1.printSchema()

root
 |-- identificador_do_processo_de_viagem: string (nullable = true)
 |-- número_da_proposta_(pcdp): string (nullable = true)
 |-- meio_de_transporte: string (nullable = true)
 |-- país__origem_ida: string (nullable = true)
 |-- uf__origem_ida: string (nullable = true)
 |-- cidade__origem_ida: string (nullable = true)
 |-- país__destino_ida: string (nullable = true)
 |-- uf__destino_ida: string (nullable = true)
 |-- cidade__destino_ida: string (nullable = true)
 |-- país__origem_volta: string (nullable = true)
 |-- uf__origem_volta: string (nullable = true)
 |-- cidade__origem_volta: string (nullable = true)
 |-- pais__destino_volta: string (nullable = true)
 |-- uf__destino_volta: string (nullable = true)
 |-- cidade__destino_volta: string (nullable = true)
 |-- valor_da_passagem: float (nullable = true)
 |-- taxa_de_serviço: float (nullable = true)
 |-- data_da_emissão/compra: string (nullable = true)
 |-- hora_da_emissão/compra: string (nullable = true)



In [18]:
df_1.show()

+-----------------------------------+-------------------------+------------------+----------------+--------------+-------------------+-----------------+----------------+-------------------+------------------+----------------+--------------------+-------------------+-----------------+---------------------+-----------------+---------------+----------------------+----------------------+
|identificador_do_processo_de_viagem|número_da_proposta_(pcdp)|meio_de_transporte|país__origem_ida|uf__origem_ida| cidade__origem_ida|país__destino_ida| uf__destino_ida|cidade__destino_ida|país__origem_volta|uf__origem_volta|cidade__origem_volta|pais__destino_volta|uf__destino_volta|cidade__destino_volta|valor_da_passagem|taxa_de_serviço|data_da_emissão/compra|hora_da_emissão/compra|
+-----------------------------------+-------------------------+------------------+----------------+--------------+-------------------+-----------------+----------------+-------------------+------------------+----------------+-

In [19]:
df_1.write.mode("overwrite").parquet(f"{dir}/2022_Passagem.parquet")

In [20]:
os.listdir(dir)

['2022_Passagem.csv', 'compras.csv', '2022_Passagem.parquet']

In [21]:
csv_size = os.path.getsize(f"{dir}/{csv_file}")
csv_size

68818335

In [24]:
parquet_size = os.path.getsize(f"{dir}/2022_Passagem.parquet")
parquet_size

4096

In [26]:
df_parquet = spark.read.parquet(f"{dir}/2022_Passagem.parquet")

df_parquet.printSchema()

root
 |-- identificador_do_processo_de_viagem: string (nullable = true)
 |-- número_da_proposta_(pcdp): string (nullable = true)
 |-- meio_de_transporte: string (nullable = true)
 |-- país__origem_ida: string (nullable = true)
 |-- uf__origem_ida: string (nullable = true)
 |-- cidade__origem_ida: string (nullable = true)
 |-- país__destino_ida: string (nullable = true)
 |-- uf__destino_ida: string (nullable = true)
 |-- cidade__destino_ida: string (nullable = true)
 |-- país__origem_volta: string (nullable = true)
 |-- uf__origem_volta: string (nullable = true)
 |-- cidade__origem_volta: string (nullable = true)
 |-- pais__destino_volta: string (nullable = true)
 |-- uf__destino_volta: string (nullable = true)
 |-- cidade__destino_volta: string (nullable = true)
 |-- valor_da_passagem: float (nullable = true)
 |-- taxa_de_serviço: float (nullable = true)
 |-- data_da_emissão/compra: string (nullable = true)
 |-- hora_da_emissão/compra: string (nullable = true)

